In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from collections import Counter
np.set_printoptions(threshold=sys.maxsize)


def generate_rule(rule_number):
    rule = np.zeros(8, dtype=int)
    binary = np.binary_repr(rule_number, width=8)
    for i in range(8):
        rule[i] = int(binary[i])
    return rule

def apply_rule(rule, neighborhood):
    decimal = np.sum(neighborhood * np.array([1, 2, 4]))
    return rule[7 - decimal]

def simulate(rule_number, num_iterations, num_cells, initial_percentage):
    np.random.seed(1)
    rule = generate_rule(rule_number)
    initial_state = np.zeros(num_cells, dtype=int)
    num_initial_cells = int(num_cells * initial_percentage / 100)
    initial_indices = np.random.choice(num_cells, num_initial_cells, replace=False)
    initial_state[initial_indices] = 1

    state = np.zeros((num_iterations, num_cells), dtype=int)
    state[0] = initial_state

    for i in range(1, num_iterations):
        for j in range(num_cells):
            neighborhood = state[i - 1, (j - 1) % num_cells], state[i - 1, j], state[i - 1, (j + 1) % num_cells]
            state[i, j] = apply_rule(rule, neighborhood)

    return state


def blockshaped(arr, n):
    assert len(arr) % n == 0, f"Length of array is not evenly divisible by {n}"
    return arr.reshape(-1, n)

def process_FHCG(state,rule_number,initial_percentage,th):
  time_steps,height = state.shape
  supercells_list = []
  new_grid_supercells = []
  # lower = 0.01
  # upper = 0.10
  # n = 10
  # range_size = (upper - lower) / n
  # boundaries = [lower + i * range_size for i in range(n+1)]
  # parts = []
  # for i in range(n):
  #   parts.append(boundaries[i])
  threshold = th
  block_size = 2
  for array in state:
    blocked_chunks = blockshaped(array,block_size)
    for each_chunk in blocked_chunks:
      supercells_list.append(list(each_chunk.flatten()))
  test_array = np.array(supercells_list)
  # print(test_array.shape)
  for i in range(len(supercells_list)):
    count = 0
    for j in range(len(supercells_list)):
      if(supercells_list[i]==supercells_list[j]):
        count = count+1 # count is number of blocks matching in the same grid (ref literature)
    new_grid_supercells.append(count/len(supercells_list))
  # print(new_grid_supercells)



  a = supercells_list
  total = len(a) # calculate the total number of nested lists in the parent list
  freq = Counter(map(tuple, a)) # count the frequency of each unique nested list in the parent list

  # calculate the probability of each unique nested list
  prob = {k: v / total for k, v in freq.items()}
  sorted_prob = dict(sorted(prob.items(), key=lambda x: x[0]))
  # print("debugging xmin: ",range(len(prob))[0])
  # print("debugging xmax: ",range(len(prob))[-1])
  # create a histogram plot of the probabilities
  plt.bar(range(len(sorted_prob)), list(sorted_prob.values()))
  plt.xticks(range(len(sorted_prob)), [int(''.join(map(str, k)), 2) for k in sorted_prob.keys()])
  plt.xlabel('Decimal Values of the binary values in block')
  plt.ylabel('Probability of the occurence')
  plt.title('Probability of occurence of supercells vs (Decimal Values)')

  # set y-axis tick locations and labels
  plt.yticks([i/20 for i in range(21)], [f'{i/20:.2f}' for i in range(21)])
  plt.ylim(0, 0.90)
  plt.hlines(threshold, xmin=range(len(sorted_prob))[0], xmax=range(len(sorted_prob))[-1], colors='r', linestyles='dashed') # drawing straight line for threshold
  # plt.hlines(lower,xmin=range(len(prob))[0],xmax=range(len(prob))[-1], colors='r', linestyles='dashed') # drawing straight line for threshold
  # plt.show()
  plt.savefig("histogram_{}_init_{}_threshold_{}.jpg".format(rule_number,initial_percentage,th))
  plt.close()
  # ################################################################
  # dynamic_states = list(np.linspace(1,0,n)).reverse()
  # for k in range(len(new_grid_supercells)):
  #   for z in range(len(thresholds)-1,0,-1):
  #     if(thresholds[z]<=new_grid_supercells[k]<=thresholds[z-1]):
  #       new_grid_supercells[k]=dynamic_states[z]
  # ################################################################

  # Simplify the threshold check
  for k in range(len(new_grid_supercells)):
      if new_grid_supercells[k] <= threshold:
          new_grid_supercells[k] = 1
      else:
          new_grid_supercells[k] = 0
  modified_board = np.array(new_grid_supercells)
  shape = int(height/block_size)
  # print(shape)
  modified_automaton = modified_board.reshape(time_steps,shape)
  return modified_automaton



def visualize(rule_number, num_iterations, num_cells, initial_percentage,th):
    state = simulate(rule_number, num_iterations, num_cells, initial_percentage)
    state_cg = process_FHCG(state,rule_number,initial_percentage,th)
    fig, ax = plt.subplots()
    ax.imshow(state, cmap='Greys', interpolation='nearest')
    # ax.set_xticks([])
    # ax.set_yticks([])
    plt.savefig(f'rule_{rule_number}_init_{initial_percentage}_threshold{th}.png')
    plt.close()
    plt.imshow(state_cg, cmap='Greys', interpolation='nearest')
    plt.savefig(f'cg_rule_{rule_number}_init_{initial_percentage}_threshold{th}.png')
    plt.close()

# Parameters
num_cells = 100
num_iterations = 100


# use this for main run
initial_percentages = [1, 10, 20, 30, 40, 50]  # Add more percentages if desired

# use this for main run
thresholds = [0.01,0.03,0.06,0.09,0.12,0.15,0.18,0.21,0.24,0.27,0.30,0.33,0.36,0.39,0.42,0.45,0.48,0.51,0.54,0.57,0.60,0.63,0.66,0.69,0.72,0.75]



# Simulate and visualize all possible 1D cellular automata rules with random initialization percentages
for rule_number in range(255,257): # left value inclusive and right value exclusive
    for initial_percentage in initial_percentages:
      for th in thresholds:
        visualize(rule_number, num_iterations, num_cells, initial_percentage,th)


<ipython-input-1-456f4f2c0a81>:10: DeprecationWarning: Insufficient bit width provided. This behavior will raise an error in the future.
  binary = np.binary_repr(rule_number, width=8)
<ipython-input-1-456f4f2c0a81>:10: DeprecationWarning: Insufficient bit width provided. This behavior will raise an error in the future.
  binary = np.binary_repr(rule_number, width=8)
<ipython-input-1-456f4f2c0a81>:10: DeprecationWarning: Insufficient bit width provided. This behavior will raise an error in the future.
  binary = np.binary_repr(rule_number, width=8)
<ipython-input-1-456f4f2c0a81>:10: DeprecationWarning: Insufficient bit width provided. This behavior will raise an error in the future.
  binary = np.binary_repr(rule_number, width=8)
<ipython-input-1-456f4f2c0a81>:10: DeprecationWarning: Insufficient bit width provided. This behavior will raise an error in the future.
  binary = np.binary_repr(rule_number, width=8)
<ipython-input-1-456f4f2c0a81>:10: DeprecationWarning: Insufficient bit wid

In [ ]:
!zip -q "image_data_$(date +'%Y%m%d%H%M%S').zip" *.jpg *.png

In [ ]:
!rm *.png
!rm *.jpg